# Анализ данных осень 2021
# Воспроизводимость и культура кода

### Почему Jupyter

1. Интерактивность необхоидма для избежания долгого перезапуска скриптов
1. Inline визуализация необходима для создания наглядных отчётов по аналитике
1. Уже интегрирован в процессы разработки больших компаний и связан с другими инструментами (spark и субд)


Почему не Google Colab

1. Долгие загрузка и обработка файлов в облаке
1. Лимит на длительность сессии
1. Версионирование colab'ов намного сложнее


Jupyter Lab

[Установка](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#pip), [документация](https://jupyterlab.readthedocs.io/en/stable/user/interface.html)

Различия с jupyter notebook:

1. Дерево файлов
1. Поддержка интерпретаторов других языков
1. Интеграция расширений от сообщества


Проблематика культуры кода в data science

1. [В 2016 году](https://arxiv.org/pdf/2003.12206.pdf) более 70% of исследователей в области data science не смогли воспроизвести результаты статей, на которые ссылались. 50% не смогли воспроизвести свои же результаты

---

### Знакомая диаграмма 

<img src="pics/bi_process.png" width="600" height="600" />

### Data Understanding

Exploratory data analysis ([EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis)) - первый этап понимания данных. Визуализация датасета и изучение

Проблемы:
1. Версионирование файлов
        raw, processed, production, experiments
1. Невоспроизводимые notebook'и
        отсутствующий код, непоследовательный запуск ячеек, отсутствие seed'ов
1. Ленивые графики, отсутствие комментариев
1. Процедурный код
1. Код других стадий
        моделирование и применение моделей в этом же ноутбуке

### Data Preparation

Подготовка данных - создание новых признаков в данных, удаление выбросов, бесполезных и недостающих значений

Проблемы:
1. Модификация существующих ноутбуков под новые модели
1. Все проблемы предыдущего этапа

### Modeling

Моделирование - создание статистических и предсказательных моделей, тренировка алгоритмов машинного обучения

Проблемы:

1. Код моделей в ноутбуке
        Не выделенный в импортируемые модули код ведёт к дублированию кода, трудностям версионирования и ревью, невоспроизводимости
1. Отсутствие записей об экспериментах
        Невозможно восстановить ход мыслей, невозможно сравнить качество новых моделей с предыдущими, сложно восстановить конфиги моделей
1. Отсутствие версионирования моделей
1. Код на Python
        Динамическая типизация, отсутствие статического анализа кода, конфликты зависимостей
1. Снова все проблемы предыдущего этапа

### Deployment

Деплой - перевод модели в production окружение, использование на новых данных

Проблемы:

1. Отсутствие запускаемого кода
        Оставшийся в ноутбуках код сложно переиспользовать
1. Множество неупорядоченных ноутбуков
1. И снова все проблемы предыдущего этапа

\+ Пример дерева ужасного проекта по анализу данных

---

## Способы решения проблем

### Хранение данных
    - В идеале - отсутствие файлов
    - Версионирование на уровне файловой системы
    - Git LFS (Large File Storage)

In [8]:
# Пример версионирования на уровне файловой системы

!tree ./examples/good_project/data/

./examples/good_project/data/
├── processed
│   ├── action2ix.pkl
│   ├── action2ix_43scr_14d_1Mu.pkl
│   ├── class_weights.pkl
│   ├── class_weights_43scr_14d_1Mu.pkl
│   ├── readme.md
│   ├── sessions_df.csv
│   └── sessions_w_time_df.csv
└── raw
    ├── events_43screens_14days_100k_users.csv
    ├── events_43screens_14days_1M_users.csv
    ├── events_43screens_14days_1M_users.csv.gz
    ├── events_62screens_1days_with_events.csv
    ├── events_62screens_1days_with_events.csv.gz
    ├── events_params.csv
    ├── readme.md
    └── sessions_1month.csv

2 directories, 15 files


### Невоспроизводимые notebook'и
    - Вынесение кода в импортируемые модули
    - Перезапуски ноутбуков при окончании работы с ними
    - Фиксирование seed'ов

In [10]:
# пример импортируемых модулей в проекте

!tree -L 2 ./examples/good_project/

./examples/good_project/
├── 00-eda.ipynb
├── 01-create_dataset.ipynb
├── 02-tf_baseline.ipynb
├── 02-train_ncsf.ipynb
├── 02-train_sess_look-alike.ipynb
├── Dockerfile
├── Pipfile
├── Pipfile.lock
├── compose-dp-updater.yml
├── data
│   ├── processed
│   └── raw
├── datasets
│   ├── __init__.py
│   ├── autoencoder.py
│   ├── filters.py
│   ├── preprocessing.py
│   └── pure.py
├── models
│   ├── __init__.py
│   ├── ffncsf
│   ├── metrics
│   └── ncsf
├── pyamplitude
│   ├── __init__.py
│   ├── amplitude_exporter.py
│   ├── amplitude_extractor.py
│   ├── config.py
│   ├── pg_saver.py
│   ├── thread_export.py
│   └── thread_extract.py
└── run_parallel_db_updater.py

9 directories, 23 files


In [11]:
# ./examples/good_project/02-tf_baseline.ipynb пример импортирования кода из модулей

# ./examples/good_project/01-create_dataset.ipynb пример перезапущенных ячеек

### Seed'ы

    - Стоит фиксировать при любых случаных процессах
    - Не стоит подбирать сиды для получения наилучших результатов

### Ленивые графики, отсутствие комментариев
    - Минимальное оформление графиков разобрано в прошлой лекции
    - Резюмирование кода в ячейке

### Процедурный код
    - Вынесение кода в модули


### Код других стадий
    - Разделение ноутбуков
    - Понимание процесса развития DS проекта

### Код моделей в ноутбуке
    - Вынесение кода в импортируемые модули, использование принципов ООП
    - Вынесение конфигов в файлы

In [12]:
# ./examples/good_project/test_settings.yml пример вынесения параметров в конфиг

### Отсутствие записей об экспериментах

   - Учёт параметров и результатов экспериментов
         
   [Omniboard](https://github.com/vivekratnavel/omniboard), [tensorboard example](https://tensorboard.dev/experiment/FRbuxfG5SkaFPQQH4OcpYw/#scalars&runSelectionState=eyJlbnNlbWJsZV9wcmVkaWN0aW9uX2JuIjp0cnVlLCJlbnNlbWJsZV90cmFpbl9ibiI6dHJ1ZSwidGVtcF9zY2FsaW5nX3ByZWRpY3Rpb25fYm4iOnRydWUsInRlbXBfc2NhbGluZ190cmFpbl9ibiI6dHJ1ZSwidmFuaWxsYV9wcmVkaWN0aW9uX2JuIjp0cnVlLCJ2YW5pbGxhX3RyYWluX2JuIjp0cnVlfQ%3D%3D)

   - Версионирование экспериментов

In [37]:
# Пример использования tensorboard

from time import sleep
import psutil
from tensorboardX import SummaryWriter

exp = 0

In [40]:
# Название эксперимента
experiment_name = 'Exp {}'.format(exp)
exp += 1

# создание логгера
logger = SummaryWriter(logdir='./examples/tensorboard_logs/' + experiment_name)

# логгирование загруженности процессора и памяти
t = 0
while 1:
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().active / 1024**3
    
    logger.add_scalar(tag='Cpu usage', scalar_value=cpu_usage, global_step=t)
    logger.add_scalar(tag='Memory usage', scalar_value=memory_usage, global_step=t)
    t += 1
    
    sleep(0.5)

KeyboardInterrupt: 

### Код на Python
   - Использование псевдотипизации
   - Использование статического анализатора [mypy](http://mypy-lang.org/)
   - Использование виртуальных сред
   - Использование другого языка внутри Python

In [47]:
# Пример использования псевдотипизации

def div2(a):
    return a / 2

print('Функция без типовых аннотаций')
print('10 / 2 =', div2(10))
print('2.5 / 2 =', div2(2.5))
print()


def div2_type_hints(a: int) -> float:
    return a / 2

print('Функция c типовыми аннотациями')
print('10 / 2 =', div2_type_hints(10))
print('2.5 / 2 =', div2_type_hints(2.5))

Функция без типовых аннотаций
10 / 2 = 5.0
2.5 / 2 = 1.25

Функция c типовыми аннотациями
10 / 2 = 5.0
2.5 / 2 = 1.25


In [49]:
# Пример более сложной псевдотипизации
from typing import List, Callable, Any

def apply_f(array: List[int], func: Callable) -> List[Any]:
    for i, item in enumerate(array):
        array[i] = func(item)
    return array

eps = 0.01
arr = [x + eps for x in range(10)]

apply_f(arr, div2_type_hints)

[0.005, 0.505, 1.005, 1.505, 2.005, 2.505, 3.005, 3.505, 4.005, 4.505]

In [51]:
# Пример проверки статической типизации

# запуск скрипта с помощью bash команды
!python3 ./examples/python_type_hints.py

[0.005, 0.505, 1.005, 1.505, 2.005, 2.505, 3.005, 3.505, 4.005, 4.505]


In [52]:
!mypy ./examples/python_type_hints.py

examples/python_type_hints.py:15: error: Argument 1 to "apply_f" has incompatible type "List[float]"; expected "List[int]"
Found 1 error in 1 file (checked 1 source file)


# Пример использования С в питоне 

https://github.com/ultrajson/ultrajson/blob/main/python/

# Виртуальные среды

Нужны для решения проблем с конфликтами зависимостей и зависимостью от окружения среды

1. Package Installer for Python (PIP) - позволяет устанавливать пакеты из общего репозитория пакетов
    Загрузка пакета репозиторий
    
    Пример команды 
    
    `pip install package`

1. Virtual Environment - устанавливает все пакеты в изолированную среду со своим python
    Пример установки пакета
    
    `python3 -m venv env_name`
    
    `source env_name/bin/activate`
    
    `pip install package`

1. Pipenv - устанавливает пакеты, контролирует конфликты зависимостей

    Пример установки пакета
    
    `pipenv install package`

In [57]:
# Пример Virtual Environment

!tree -L 1 ./examples/venv/

./examples/venv/
├── bin
├── include
├── lib
└── pyvenv.cfg

3 directories, 1 file


In [61]:
# Пример Pipenv

!ls -l ./examples/good_project/*Pipfile*

-rw-r--r--  1 i.shamov  staff    269 16 сен 17:16 ./examples/good_project/Pipfile
-rw-r--r--  1 i.shamov  staff  20161 16 сен 14:47 ./examples/good_project/Pipfile.lock


### Множество неупорядоченных ноутбуков
    - Использовать нумерацию в названиях файлов ноутбуков

---

Короткое знакомство с [Kaggle](kaggle.com)